In [ ]:
import pandas as pd
import numpy as np
from time import time
from Project import Info

import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials

# Chaves de acesso vinculadas à conta do Spotify
# foram removidas para preservar a segurança da conta
# %env SPOTIPY_CLIENT_ID = 
# %env SPOTIPY_CLIENT_SECRET = 

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [20]:
# Pesquisando pelo ID de uma música, dado seu nome e o nome do artista
artista = "Haken"
musica  = "Atlas Stone"
track_id = sp.search(q = f'artist:{artista} track:{musica} ', type = 'track')
track_id = track_id['tracks']['items'][0]['id']
# Extraindo os atributos do áudio, dado o id da música
sp.audio_features(track_id)[0]

{'danceability': 0.45,
 'energy': 0.896,
 'key': 0,
 'loudness': -5.869,
 'mode': 1,
 'speechiness': 0.0455,
 'acousticness': 0.00435,
 'instrumentalness': 0.00124,
 'liveness': 0.145,
 'valence': 0.34,
 'tempo': 105.082,
 'type': 'audio_features',
 'id': '0ZkkO456qIrAH9cr9DcPlW',
 'uri': 'spotify:track:0ZkkO456qIrAH9cr9DcPlW',
 'track_href': 'https://api.spotify.com/v1/tracks/0ZkkO456qIrAH9cr9DcPlW',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ZkkO456qIrAH9cr9DcPlW',
 'duration_ms': 453867,
 'time_signature': 3}

In [21]:
def get_audio_features(track_id: str) -> np.array:
    '''
    Extrai as informações de interesse do dicionário retornado pela função audio_features().
    '''
    result = sp.audio_features(track_id)[0]
    return np.array([result['danceability'], result['energy'], result['key'], result['loudness'], result['mode'], result['speechiness'], result['acousticness'], 
                     result['instrumentalness'], result['liveness'], result['valence'], result['tempo'],result['duration_ms'], result['time_signature']]) \
          if result != None else None

In [22]:
get_audio_features(track_id)

array([ 4.50000e-01,  8.96000e-01,  0.00000e+00, -5.86900e+00,
        1.00000e+00,  4.55000e-02,  4.35000e-03,  1.24000e-03,
        1.45000e-01,  3.40000e-01,  1.05082e+02,  4.53867e+05,
        3.00000e+00])

In [23]:
pl_id = 'spotify:playlist:5RIbzhG2QqdkaP24iXLnZX'
offset = 0
tracks = []
while True:
    response = sp.playlist_items(pl_id,
                                 offset=offset,
                                 fields='items.track.id,total',
                                 additional_types=['track'])
    
    if len(response['items']) == 0:
        break
    
    for track in response['items']:
        tracks.append(track['track']['id'])
        
    offset = offset + len(response['items'])
    print(offset, "/", response['total'])

100 / 11000
200 / 11000
300 / 11000
400 / 11000
500 / 11000
600 / 11000
700 / 11000
800 / 11000
900 / 11000
1000 / 11000
1100 / 11000
1200 / 11000
1300 / 11000
1400 / 11000
1500 / 11000
1600 / 11000
1700 / 11000
1800 / 11000
1900 / 11000
2000 / 11000
2100 / 11000
2200 / 11000
2300 / 11000
2400 / 11000
2500 / 11000
2600 / 11000
2700 / 11000
2800 / 11000
2900 / 11000
3000 / 11000
3100 / 11000
3200 / 11000
3300 / 11000
3400 / 11000
3500 / 11000
3600 / 11000
3700 / 11000
3800 / 11000
3900 / 11000
4000 / 11000
4100 / 11000
4200 / 11000
4300 / 11000
4400 / 11000
4500 / 11000
4600 / 11000
4700 / 11000
4800 / 11000
4900 / 11000
5000 / 11000
5100 / 11000
5200 / 11000
5300 / 11000
5400 / 11000
5500 / 11000
5600 / 11000
5700 / 11000
5800 / 11000
5900 / 11000
6000 / 11000
6100 / 11000
6200 / 11000
6300 / 11000
6400 / 11000
6500 / 11000
6600 / 11000
6700 / 11000
6800 / 11000
6900 / 11000
7000 / 11000
7100 / 11000
7200 / 11000
7300 / 11000
7400 / 11000
7500 / 11000
7600 / 11000
7700 / 11000
7800 / 1

In [24]:
tracks = np.array(tracks).reshape(-1, 1)

In [26]:
start = time()
attributes = np.apply_along_axis(get_audio_features, 1, tracks)
end = time()
print(f"Elapsed Time: {(end - start)/60} min")

Elapsed Time: 29.0332386692365 min


In [31]:
temp = pd.DataFrame(attributes, columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', \
                                           'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'])
temp

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.680,0.853,11.0,-10.153,0.0,0.0528,0.270000,0.962,0.1640,0.9330,110.980,543999.0,4.0
1,0.660,0.961,8.0,-7.299,0.0,0.0424,0.003660,0.868,0.1120,0.5380,132.999,329000.0,3.0
2,0.725,0.961,4.0,-6.104,0.0,0.0452,0.000008,0.864,0.1270,0.0745,132.002,383953.0,4.0
3,0.725,0.951,7.0,-6.329,1.0,0.0729,0.014600,0.742,0.1090,0.3860,132.988,461955.0,4.0
4,0.798,0.448,1.0,-11.935,1.0,0.1300,0.068600,0.827,0.0859,0.3800,122.028,377705.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,0.574,0.964,2.0,-8.982,1.0,0.0523,0.000965,0.795,0.1380,0.6360,144.214,320663.0,4.0
10996,0.649,0.552,7.0,-14.186,1.0,0.0374,0.521000,0.922,0.1250,0.0293,127.897,311209.0,4.0
10997,0.931,0.485,2.0,-11.594,1.0,0.1010,0.638000,0.729,0.0922,0.2340,110.062,414227.0,4.0
10998,0.603,0.768,8.0,-11.510,0.0,0.0337,0.002820,0.903,0.1340,0.0881,110.749,376116.0,4.0
